In [1]:
import ROOT
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
ROOT.ROOT.EnableImplicitMT()

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/func_marti.cc","k")

if "/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/functions.so" not in ROOT.gSystem.GetLibraries():
    ROOT.gSystem.CompileMacro("/home/submit/pdmonte/CMSSW_10_6_27/src/Hrare2023/analysis/functions.cc","k")

Welcome to JupyROOT 6.28/00


In [6]:
date = "JUN21"

chain = ROOT.TChain("events")
chain.Add("/data/submit/pdmonte/outputs/{0}/2018/outname_mc1041_GFcat_D0StarCat_2018.root".format(date))

1

In [24]:
df_SGN = ROOT.RDataFrame(chain)
df_SGN = df_SGN.Define("scale", "w*lumiIntegrated")\
            .Define("filter1", "abs(goodMeson_eta) > 1.4")\
            .Define("filter2", "Vec_i {static_cast<int>(getPt(GenPart_pt, GenPart_pdgId, GenPart_genPartIdxMother, 22, 423, 25).size())}")\
            .Define("filter3", "filter2 == 1")

In [25]:
cols = ["goodMeson_ditrk_pt", "filter1", "filter2", "filter3"]

x = df_SGN.AsNumpy(columns=cols)

pddf = pd.DataFrame(x)

display(pddf)

print(type(pddf.iloc[0]["goodMeson_ditrk_pt"]))
print(type(pddf.iloc[0]["filter1"]))
print(type(pddf.iloc[0]["filter2"]))
print(type(pddf.iloc[0]["filter3"]))

,goodMeson_ditrk_pt,filter1,filter2,filter3
0,[53.76828384399414],[1],[0],[0]
1,[57.202308654785156],[1],[0],[0]
2,[172.90272521972656],[1],[1],[1]
3,[53.86904525756836],[1],[0],[0]
4,[54.38526916503906],[1],[0],[0]
...,...,...,...,...
47217,[47.53954315185547],[1],[0],[0]
47218,[61.203582763671875],[1],[0],[0]
47219,[65.57756805419922],[0],[1],[1]
47220,[121.49837493896484],[0],[0],[0]


<class cppyy.gbl.ROOT.VecOps.RVec<float> at 0x564b8da1f330>
<class cppyy.gbl.ROOT.VecOps.RVec<int> at 0x564b8e47dee0>
<class cppyy.gbl.ROOT.VecOps.RVec<int> at 0x564b8e47dee0>
<class cppyy.gbl.ROOT.VecOps.RVec<int> at 0x564b8e47dee0>
